In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from datetime import date
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.stats import pearsonr
from sklearn import preprocessing
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.weightstats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel(r"/Resultados Finales_INCO 2021.xlsx")

df_copia = df.copy()

In [3]:
aux_v1=[]

for i in range(len(df)):
    if (df.loc[i,'RANGO_PUNTAJE']== 'Muy alto'):
        resultado = 0
    elif (df.loc[i,'RANGO_PUNTAJE']== 'Alto'):
        resultado = 0
    else:
        resultado = 1   

    aux_v1.append(resultado)  

In [4]:
numerical_df = df.select_dtypes(include=[np.number])
numerical_df

,COD ENTIDAD,Sanciones contra funcionarios y servidores públicos en el RNSSC,Funcionarios y servidores públicos con responsabilidad administrativa,Contrataciones sin competencia,Presunto fraccionamiento en las contrataciones sin proceso (hasta montos menores o iguales a 8 UIT),Ejecutoras con proyectos caducos y abandonados,Nivel de Madurez de SCI (inferior al nivel básico),Responsabilidades administrativas sin sancionar,Funcionarios y servidores públicos que no declaran a sus parientes,Funcionarios y servidores públicos que no declaran información bancaria,...,Cantidad de servicios de control,Contrataciones que son materia de impedimentos por la Ley de Contrataciones del Estado.,Presuntas irregularidades en la gestión de planillas,PUNTAJE TOTAL,DIM 1,V1,V2,DIM 2,V3,V4
0,434,6.000000,5.0,3.000000,5.444444,6.000000,4,1.000000,4.000000,3.666667,...,5.000000,10,0,85.4,40.444444,29.444444,11.000000,45.000000,15.0,30.000000
1,5458,4.888889,5.0,2.333333,5.444444,6.000000,4,1.333333,1.666667,1.444444,...,5.000000,7,10,85.0,35.111111,27.666667,7.444444,49.911111,13.8,36.111111
2,5345,4.888889,5.0,3.000000,6.000000,3.777778,2,1.111111,4.000000,5.000000,...,5.000000,10,3,84.5,38.111111,24.666667,13.444444,46.355556,13.8,32.555556
3,5332,6.000000,5.0,2.111111,5.444444,6.000000,0,2.000000,3.333333,2.777778,...,5.000000,9,7,83.2,33.888889,24.555556,9.333333,49.355556,13.8,35.555556
4,5350,5.444444,5.0,2.111111,6.000000,6.000000,0,1.222222,3.666667,2.777778,...,5.000000,10,3,83.0,36.222222,24.555556,11.666667,46.800000,13.8,33.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,3358,0.000000,0.0,2.333333,0.000000,0.000000,0,0.000000,0.000000,1.000000,...,1.000000,0,0,5.8,3.333333,2.333333,1.000000,2.444444,0.0,2.444444
2802,6451,0.000000,0.0,2.555556,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0,1,5.6,4.555556,2.555556,2.000000,1.000000,0.0,1.000000
2803,4999,0.000000,0.0,0.000000,1.000000,0.000000,0,0.000000,0.000000,0.000000,...,1.444444,1,0,5.4,3.000000,1.000000,2.000000,2.444444,0.0,2.444444
2804,838,0.000000,0.0,0.000000,0.000000,1.000000,0,0.000000,0.000000,0.000000,...,1.000000,1,0,5.0,3.000000,1.000000,2.000000,2.000000,0.0,2.000000


# MRL con 0 y 1 sin PCA 

In [5]:
df['RANGO_PUNTAJE'] = aux_v1
numerical_df = df.select_dtypes(include=[np.number])
drop_columnas = ['RANGO_PUNTAJE']
X = numerical_df.drop(drop_columnas, axis=1)
y = df['RANGO_PUNTAJE']
X_train, X_test, y_train, y_test = train_test_split( X,y,train_size= 0.8,random_state = 0,shuffle= True)

lr = linear_model.LogisticRegression(C=1000,random_state=1)
lr.fit(X_train,y_train)
    
y_pred= lr.predict(X_test)
m_c = pd.crosstab(y_pred, y_test, rownames=['Predicho'], colnames=['Observado'])
Acc= accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
Esp =m_c.iloc[0,0]/(m_c.iloc[0,0]+m_c.iloc[1,0])
Sens = m_c.iloc[1,1]/(m_c.iloc[0,1]+m_c.iloc[1,1])

In [6]:
print('Modelo:',lr)
print(f"Accuracy:{round(Acc*100,2)}%")
print(f"Specificity:{round(Esp*100,2)}%")
print(f"Sensitivity:{round(Sens*100,2)}%")


Modelo: LogisticRegression(C=1000, random_state=1)
Accuracy:95.73%
Specificity:56.25%
Sensitivity:98.11%


In [7]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute error :", mae)
print(f"Mean Squared  error :", mse)
print(f"Mean Mean Squared error :", rmse)


Mean Absolute error : 0.042704626334519574
Mean Squared  error : 0.042704626334519574
Mean Mean Squared error : 0.2066509770954872


# MRL con 0, 1, -1 sin PCA 

In [8]:
df_2 = df_copia
df_2['RANGO_PUNTAJE'].unique()

array(['Muy alto', 'Alto', 'Medio alto', 'Medio', 'Moderado', 'Bajo'],
      dtype=object)

In [9]:
aux_v1=[]

for i in range(len(df_2)):
    if ((df_2.loc[i,'RANGO_PUNTAJE']== 'Muy alto')or (df_2.loc[i,'RANGO_PUNTAJE']== 'Alto')):
        resultado = 1
    elif ((df_2.loc[i,'RANGO_PUNTAJE']== 'Medio alto'or df_2.loc[i,'RANGO_PUNTAJE']== 'Medio')):
        resultado = 0
    else:
        resultado = -1   

    aux_v1.append(resultado) 
df_2['RANGO_PUNTAJE'] = aux_v1

In [10]:
value_counts = df_2['RANGO_PUNTAJE'].value_counts()
value_counts

RANGO_PUNTAJE
-1    1771
 0     863
 1     172
Name: count, dtype: int64

In [11]:
df_2['RANGO_PUNTAJE'] = aux_v1
numerical_df = df_2.select_dtypes(include=[np.number])
drop_columnas = ['RANGO_PUNTAJE']
X = numerical_df.drop(drop_columnas, axis=1)
y = df_2['RANGO_PUNTAJE']
X_train, X_test, y_train, y_test = train_test_split( X,y,train_size= 0.8,random_state = 0,shuffle= True)

lr = linear_model.LogisticRegression(C=1000,random_state=1)
lr.fit(X_train,y_train)
    
y_pred= lr.predict(X_test)
m_c = pd.crosstab(y_pred, y_test, rownames=['Predicho'], colnames=['Observado'])
Acc= accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
Esp =m_c.iloc[0,0]/(m_c.iloc[0,0]+m_c.iloc[1,0])
Sens = m_c.iloc[1,1]/(m_c.iloc[0,1]+m_c.iloc[1,1])

In [12]:
print('Modelo:',lr)
print(f"Accuracy:{round(Acc*100,2)}%")
print(f"Specificity:{round(Esp*100,2)}%")
print(f"Sensitivity:{round(Sens*100,2)}%")

Modelo: LogisticRegression(C=1000, random_state=1)
Accuracy:79.54%
Specificity:91.47%
Sensitivity:77.85%


In [13]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute error : {round(mae*100,2)}%")
print(f"Mean Squared  error :{round(mse*100,2)}%" )
print(f"Mean Mean Squared error :{round(rmse*100)}%")

Mean Absolute error : 21.17%
Mean Squared  error :22.6%
Mean Mean Squared error :48%


# MRL con -1 0 1 con PCA y sin muestreo aleatorio


In [14]:
df_2.head(1)

,COD ENTIDAD,ENTIDAD,DEPARTAMENTO,PROVINCIA,DISTRITO,NIVEL DE GOBIERNO,Sanciones contra funcionarios y servidores públicos en el RNSSC,Funcionarios y servidores públicos con responsabilidad administrativa,Contrataciones sin competencia,Presunto fraccionamiento en las contrataciones sin proceso (hasta montos menores o iguales a 8 UIT),...,Presuntas irregularidades en la gestión de planillas,PUNTAJE TOTAL,DIM 1,V1,V2,DIM 2,V3,V4,MACRORREGIÓN,RANGO_PUNTAJE
0,434,MUNICIPALIDAD METROPOLITANA DE LIMA,LIMA METROPOLITANA,LIMA METROPOLITANA,LIMA,GOBIERNO LOCAL,6.0,5.0,3.0,5.444444,...,0,85.4,40.444444,29.444444,11.0,45.0,15.0,30.0,LIMA Y CALLAO,1


In [15]:
numerical_df = df_2.select_dtypes(include=[np.number])
drop = ['RANGO_PUNTAJE']
Y = numerical_df['RANGO_PUNTAJE']
X = numerical_df.drop(drop, axis=1)

In [16]:
#PCA analisis
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)
pca = PCA(n_components=3) 
principal_components = pca.fit_transform(scaled_df )
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
loading_scores = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2', 'PC3'])
pca_df


,PC1,PC2,PC3
0,11.179056,-2.010971,2.023977
1,10.232193,0.186696,4.031564
2,10.923992,0.217874,1.700242
3,11.057830,1.202365,1.310723
4,11.083409,0.023564,1.307895
...,...,...,...
2801,-4.366424,1.486734,-1.975051
2802,-4.838312,1.851853,-0.846776
2803,-4.526864,1.206911,-2.446068
2804,-4.627682,0.210892,-2.898869


In [17]:
X = pca_df
X_train, X_test, y_train, y_test = train_test_split( X,Y,train_size= 0.8,random_state = 0,shuffle= True)

lr = linear_model.LogisticRegression(C=1000,random_state=1)
lr.fit(X_train,y_train)
    
y_pred= lr.predict(X_test)
m_c = pd.crosstab(y_pred, y_test, rownames=['Predicho'], colnames=['Observado'])
Acc= accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
Esp =m_c.iloc[0,0]/(m_c.iloc[0,0]+m_c.iloc[1,0])
Sens = m_c.iloc[1,1]/(m_c.iloc[0,1]+m_c.iloc[1,1])

In [18]:
print('Modelo:',lr)
print(f"Accuracy:{round(Acc*100,2)}%")
print(f"Specificity:{round(Esp*100,2)}%")
print(f"Sensitivity:{round(Sens*100,2)}%")

Modelo: LogisticRegression(C=1000, random_state=1)
Accuracy:96.26%
Specificity:97.97%
Sensitivity:94.02%


In [19]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute error : {round(mae*100,2)}%")
print(f"Mean Squared  error :{round(mse*100,2)}%" )
print(f"Mean Mean Squared error :{round(rmse*100)}%")

Mean Absolute error : 3.74%
Mean Squared  error :3.74%
Mean Mean Squared error :19%


## Modelo con undersampling (submuestreo)

In [20]:
df_2['RANGO_PUNTAJE'] = aux_v1
numerical_df = df_2.select_dtypes(include=[np.number])
drop_columnas = ['RANGO_PUNTAJE']
X = numerical_df.drop(drop_columnas, axis=1)

#PCA analisis
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)
len(scaled_df)
pca = PCA(n_components=5) 
principal_components = pca.fit_transform(scaled_df )
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3','PC4', 'PC5'])
loading_scores = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2', 'PC3','PC4', 'PC5'])



In [21]:
X = pca_df
y = numerical_df['RANGO_PUNTAJE']
len(y)
X_train, X_test, y_train, y_test = train_test_split( X,y,train_size= 0.8,random_state = 0,shuffle= True)


In [22]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)                                            
Xunder, yunder = rus.fit_resample(X_train,y_train)
print(pd.DataFrame(yunder.value_counts()))

               count
RANGO_PUNTAJE       
-1               140
 0               140
 1               140


In [23]:
lr = linear_model.LogisticRegression(C=1000,random_state=1)
lr.fit(Xunder,yunder)
    
y_pred= lr.predict(X_test)
m_c = pd.crosstab(y_pred, y_test, rownames=['Predicho'], colnames=['Observado'])
Acc= accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
Esp =m_c.iloc[0,0]/(m_c.iloc[0,0]+m_c.iloc[1,0])
Sens = m_c.iloc[1,1]/(m_c.iloc[0,1]+m_c.iloc[1,1])

In [24]:
print('Modelo:',lr)
print(f"Accuracy:{round(Acc*100,2)}%")
print(f"Specificity:{round(Esp*100,2)}%")
print(f"Sensitivity:{round(Sens*100,2)}%")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute error : {round(mae*100,2)}%")
print(f"Mean Squared  error :{round(mse*100,2)}%" )
print(f"Mean Mean Squared error :{round(rmse*100)}%")

Modelo: LogisticRegression(C=1000, random_state=1)
Accuracy:94.66%
Specificity:95.64%
Sensitivity:95.56%
Mean Absolute error : 5.34%
Mean Squared  error :5.34%
Mean Mean Squared error :23%


# Modelo con oversampling (sobremuestreo)

In [25]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
ros = RandomOverSampler(random_state=1234)                                              #Inicializar
xover,yover = ros.fit_resample(X_train,y_train)
print(pd.DataFrame(yover.value_counts()))

               count
RANGO_PUNTAJE       
-1              1427
 0              1427
 1              1427


In [26]:
lr = linear_model.LogisticRegression(C=1000,random_state=1)
lr.fit(xover,yover)
    
y_pred= lr.predict(X_test)
m_c = pd.crosstab(y_pred, y_test, rownames=['Predicho'], colnames=['Observado'])
Acc= accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
Esp =m_c.iloc[0,0]/(m_c.iloc[0,0]+m_c.iloc[1,0])
Sens = m_c.iloc[1,1]/(m_c.iloc[0,1]+m_c.iloc[1,1])

In [27]:
print('Modelo:',lr)
print(f"Accuracy:{round(Acc*100,2)}%")
print(f"Specificity:{round(Esp*100,2)}%")
print(f"Sensitivity:{round(Sens*100,2)}%")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Absolute error : {round(mae*100,2)}%")
print(f"Mean Squared  error :{round(mse*100,2)}%" )
print(f"Mean Mean Squared error :{round(rmse*100)}%")

Modelo: LogisticRegression(C=1000, random_state=1)
Accuracy:96.62%
Specificity:97.97%
Sensitivity:97.22%
Mean Absolute error : 3.38%
Mean Squared  error :3.38%
Mean Mean Squared error :18%
